In [ ]:
import ee
from geetools import tools, ui, collection, utils
import pandas as pd

## Get a NDVI Image

In [2]:
l8 = collection.landsat.Landsat(8)

In [3]:
p = ee.Geometry.Point([-72,-42])

In [4]:
col = l8.collection.filterBounds(p)

In [5]:
i = ee.Image(col.first())

In [6]:
ndvi = l8.ndvi(i)

## Get random points

In [7]:
rani = ee.Image.random(1).clip(ndvi.geometry()).clip(ndvi.geometry())

In [8]:
rani_thres = rani.lte(0.000001)
pixcoord = ee.Image.pixelLonLat().updateMask(rani_thres)
values = pixcoord.reduceRegion(reducer=ee.Reducer.toList(), geometry=ndvi.geometry(), scale=30, maxPixels=1e13)
point_list = ee.List(values.get('longitude')).zip(values.get('latitude'))
points = ee.FeatureCollection(point_list.map(lambda coord: ee.Feature(ee.Geometry.Point(coord))))

# Linear Distribution

In [20]:
d = tools.image.distribution_linear(ndvi, 'ndvi', mean=0.3, range_min=0, range_max=1, min=2, max=5)
result = ndvi.addBands(d)

In [21]:
chart = ui.chart.Image.bandsByRegion(result, points.limit(100), xProperty='ndvi')

# Normal Distribution

In [31]:
normal = tools.image.distribution_normal(ndvi, 'ndvi', mean=0.3, range_min=0, range_max=1, min=2, max=5)
result_n = ndvi.addBands(normal)

In [32]:
chart_normal = ui.chart.Image.bandsByRegion(result_n, points.limit(100), xProperty='ndvi', bands=['normal_dist'])

# Both Charts

In [33]:
both = chart_normal.cat(chart)

In [34]:
both.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [35]:
both.dataframe

,linear_dist,normal_dist
-0.012388,3.661193,3.374267
-0.007408,3.682535,3.408623
-0.002468,3.703709,3.442991
-0.002456,3.703760,3.443073
-0.001964,3.705868,3.446511
-0.000094,3.713885,3.459601
0.000322,3.715666,3.462514
0.002173,3.723598,3.475510
0.003397,3.728842,3.484122
0.003675,3.730036,3.486083
